### все импорты

In [31]:
! pip install dataframe_image

In [1]:
import os, json
import pandas as pd
import numpy as np

from tqdm import tqdm
from datetime import datetime
import time

from sklearn import preprocessing  # там OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import ensemble
from sklearn import metrics
import pickle
import seaborn as sns

import dataframe_image as dfi

from sklearn.neighbors import KNeighborsClassifier


In [2]:
big_df = pd.read_csv('big_df2_most_recent_features.tsv', sep='\t')  

In [3]:
big_df.columns

Index(['Role', 'Rank', 'Last_word', 'pos', 'case', 'gender', 'animacy',
       'number', 'lemma', 'link_name', 't', 'r', 'left_context',
       'right_context'],
      dtype='object')

In [7]:
pd.DataFrame(big_df.groupby('Role')['lemma'].count()).head(40)

,lemma
Role,
агенс,1194
адресат,238
атрибут,20
валидность,1
вводное,1
вероятность,2
возможность,1
время,703
дейксис,1


In [54]:
big_df[big_df['Role'] == 'время']['lemma'].value_counts()

год           129
время          60
день           44
раз            39
неделя         24
             ... 
джентльмен      1
слово           1
путь            1
курс            1
заседание       1
Name: lemma, Length: 171, dtype: int64

In [6]:
pd.DataFrame(big_df['Role'].value_counts()).head(20)

,Role
пациенс,7813
место,3114
агенс,1194
инструмент,859
причина,807
признак,804
время,703
экспериенцер,426
адресат,238
цель,218


### multi class classification

encode features

In [8]:
def features(df, target):
        """
        отдает только те колонки датафрейма, которые
        не таргет колонка
        categorical columns -> one-hot encoding
        для некатегориальных строковых используется TfidfVectorizer
        """
        my_features = df.drop(target, axis='columns')
        my_features = my_features.fillna('unknown')
        cols = my_features.columns

        
        non_categ = []
        categorical_columns = []
        
        vectorizer = TfidfVectorizer()

        for col in cols:

            percent = len(my_features[col].unique())/len(my_features[col])
            
            if percent < 0.1:
                
                if type(my_features[col].tolist()[0]) == str:
                    #  to one hot columns
                    categorical_columns.append(col)
            else:
                # векторизация
                if type(my_features[col].tolist()[0]) == str:
                    
                    vectors = vectorizer.fit_transform(my_features[col])
                    vectors = vectors.toarray()
                    columns_for_df = vectorizer.get_feature_names()
                    df = pd.DataFrame(vectors, columns=columns_for_df)
                    df = df.add_prefix(col+'_')
                    non_categ += list(df.columns)
                    my_features = my_features.join(df)
                    
                    # убрать исходную невекторизованную колонку
                    my_features = my_features.drop([col], axis=1)
                    
                    
        if categorical_columns:  # list with names of categ cols
            
            enc = preprocessing.OneHotEncoder(handle_unknown='ignore')
            train_X_encoded = enc.fit_transform(my_features[categorical_columns]).toarray()
            new_col_names = enc.get_feature_names(categorical_columns)
            train_X_encoded =  pd.DataFrame(train_X_encoded, columns=new_col_names)
            
            my_features = pd.concat([my_features[non_categ],train_X_encoded], axis=1)
            
        return my_features

In [9]:
def get_X_Y(df, enc_features):
    le = preprocessing.LabelEncoder()
    le.fit(df['Role'])
    print(le.classes_) #
    y = le.transform(df['Role'])
    x = enc_features.to_numpy()
    return x, y, le.classes_

In [71]:
def find_optimal_sample_num(big_df2):
    list_of_sampled_df = []
    for num in range(100, 900, 100):
        counts = pd.DataFrame(big_df2['Role'].value_counts())
        counts.rename(columns={'Role': 'Count'}, inplace=True)
        counts = counts[counts['Count'] > num]
        to_take = list(counts.index)
        res = pd.DataFrame([])
        for role in to_take:
            role_samples = big_df2[big_df2['Role'] == role].sample(n=num, random_state=1)
            res = res.append(role_samples, ignore_index=True)
        list_of_sampled_df.append((res, num))
    
    return list_of_sampled_df

report for different sample size for random forest

In [74]:
def collect_f1_rand_forest(list_of_sampled_df):
    df_report = pd.DataFrame([])
    for df, num in tqdm(list_of_sampled_df):
        num = str(num)
        enc_features = features(df, 'Role')
        x, y, le_classes = get_X_Y(df, enc_features)
        train_x, test_x, train_y, test_y = train_test_split(x, y, \
                                                        test_size=0.3, \
                                                        random_state=42)
        mod = ensemble.RandomForestClassifier(random_state=3, n_estimators=10, n_jobs=-1)
        report = model(mod, train_x, train_y, test_x, test_y, le_classes=le_classes)
        df_report = pd.concat([df_report, pd.DataFrame(report).transpose().add_prefix(num+'_')[num+'_f1-score']], axis=1)
    return df_report 

### 1. RandomForestClassifier

In [14]:
def model(model, train_x, train_y, test_x, test_y, le_classes):
    model.fit(train_x, train_y)
    report = metrics.classification_report(test_y, model.predict(test_x),
                                  target_names=le_classes, output_dict=True)
    return report

In [160]:
list_of_sampled_df = find_optimal_sample_num(big_df)

In [161]:
len(list_of_sampled_df)

10

In [165]:
all_report = collect_f1_rand_forest(list_of_sampled_df)


  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

['агенс' 'адресат' 'время' 'инструмент' 'место' 'пациенс' 'признак'
 'причина' 'ситуация в фокусе' 'цель' 'экспериенцер']



 10%|████████▎                                                                          | 1/10 [00:02<00:22,  2.48s/it]

['агенс' 'адресат' 'время' 'инструмент' 'место' 'пациенс' 'признак'
 'причина' 'цель' 'экспериенцер']



 20%|████████████████▌                                                                  | 2/10 [00:07<00:25,  3.13s/it]

...


 80%|██████████████████████████████████████████████████████████████████▍                | 8/10 [02:25<00:54, 27.12s/it]

['агенс' 'место' 'пациенс']



 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [02:39<00:23, 23.12s/it]

['место' 'пациенс']



100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [04:07<00:00, 24.71s/it]


### without context

In [26]:
cols = list(big_df.columns)
cols.remove('Last_word')

In [27]:
print(cols)

['Role', 'Rank', 'pos', 'case', 'gender', 'animacy', 'number', 'lemma', 'link_name', 't', 'r', 'left_context', 'right_context']


не убирая контекст

In [72]:
def with_context(big_df):
    list_of_sampled_df = find_optimal_sample_num(big_df)
    f1s = collect_f1_rand_forest(list_of_sampled_df)
#     overfit_check_f1s = check_overfit(list_of_sampled_df)
        
    cm = sns.light_palette("green", as_cmap=True)
    styled = f1s.style.background_gradient(cmap=cm)\
    .set_caption("100-800 sample size with context")\
    .set_table_styles([{
        'selector': 'caption',
        'props': [
            ('color', 'black'),
            ('font-size', '16px')
        ]
    }])
    
    dfi.export(styled, 'with_context_f1s.png')
        
#     cm = sns.light_palette("orange", as_cmap=True)
#     styled = overfit_check_f1s.style.background_gradient(cmap=cm)\
#     .set_caption("100-800 sample size without context")\
#     .set_table_styles([{
#         'selector': 'caption',
#         'props': [
#             ('color', 'black'),
#             ('font-size', '16px')
#         ]
#     }])   
#     dfi.export(styled, 'without_context_f1s_overfit_check.png')

In [75]:
with_context(big_df)

  0%|                                                                                            | 0/8 [00:00<?, ?it/s]

['агенс' 'адресат' 'время' 'инструмент' 'место' 'пациенс' 'признак'
 'причина' 'ситуация в фокусе' 'цель' 'экспериенцер']


 12%|██████████▌                                                                         | 1/8 [00:02<00:15,  2.24s/it]

['агенс' 'адресат' 'время' 'инструмент' 'место' 'пациенс' 'признак'
 'причина' 'цель' 'экспериенцер']


 25%|█████████████████████                                                               | 2/8 [00:11<00:26,  4.35s/it]

...

 75%|███████████████████████████████████████████████████████████████                     | 6/8 [01:45<00:42, 21.11s/it]

['агенс' 'время' 'инструмент' 'место' 'пациенс' 'признак' 'причина']


 88%|█████████████████████████████████████████████████████████████████████████▌          | 7/8 [02:31<00:28, 28.38s/it]

['агенс' 'инструмент' 'место' 'пациенс' 'признак' 'причина']


100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [03:48<00:00, 28.55s/it]


что если убрать левый/правый контекст

In [33]:
def whats_with_context(big_df):
    res = []
    for col in ['left_context', 'right_context']:
        list_of_sampled_df = find_optimal_sample_num(big_df.drop([col], axis='columns'))
        f1s = collect_f1_rand_forest(list_of_sampled_df)
        overfit_check_f1s = check_overfit(list_of_sampled_df)
        
        cm = sns.light_palette("green", as_cmap=True)
        print("\nWithout ", col)
#         print(f1s.style.background_gradient(cmap=cm))
        styled = f1s.style.background_gradient(cmap=cm)\
        .set_caption("100-800 sample size without " + col)\
        .set_table_styles([{
            'selector': 'caption',
            'props': [
                ('color', 'black'),
                ('font-size', '16px')
            ]
        }])
    
        dfi.export(styled, col+'_without_f1s.png')
        
        cm = sns.light_palette("orange", as_cmap=True)
        
        styled = overfit_check_f1s.style.background_gradient(cmap=cm)\
        .set_caption("100-800 sample size without " + col)\
        .set_table_styles([{
            'selector': 'caption',
            'props': [
                ('color', 'black'),
                ('font-size', '16px')
            ]
        }])
        
        dfi.export(styled, col+'_without_f1s_overfit_check.png')
    
    return res

In [34]:
whats_with_context(big_df)

  0%|                                                                                            | 0/8 [00:00<?, ?it/s]

['агенс' 'адресат' 'время' 'инструмент' 'место' 'пациенс' 'признак'
 'причина' 'ситуация в фокусе' 'цель' 'экспериенцер']


 12%|██████████▌                                                                         | 1/8 [00:00<00:06,  1.01it/s]

['агенс' 'адресат' 'время' 'инструмент' 'место' 'пациенс' 'признак'
 'причина' 'цель' 'экспериенцер']


 25%|█████████████████████                                                               | 2/8 [00:03<00:08,  1.38s/it]

...

['агенс' 'время' 'инструмент' 'место' 'пациенс' 'признак' 'причина']


 88%|█████████████████████████████████████████████████████████████████████████▌          | 7/8 [00:41<00:07,  7.74s/it]

['агенс' 'инструмент' 'место' 'пациенс' 'признак' 'причина']


100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [01:02<00:00,  7.82s/it]



Without  right_context


что если убрать контекст вообще

In [38]:
def whats_with_context2(big_df):
    list_of_sampled_df = find_optimal_sample_num(big_df.drop(['left_context', 'right_context'], axis='columns'))
    f1s = collect_f1_rand_forest(list_of_sampled_df)
    overfit_check_f1s = check_overfit(list_of_sampled_df)
        
    cm = sns.light_palette("green", as_cmap=True)
    styled = f1s.style.background_gradient(cmap=cm)\
    .set_caption("100-800 sample size without context")\
    .set_table_styles([{
        'selector': 'caption',
        'props': [
            ('color', 'black'),
            ('font-size', '16px')
        ]
    }])
    
    dfi.export(styled, 'without_context_f1s.png')
        
    cm = sns.light_palette("orange", as_cmap=True)
    styled = overfit_check_f1s.style.background_gradient(cmap=cm)\
    .set_caption("100-800 sample size without context")\
    .set_table_styles([{
        'selector': 'caption',
        'props': [
            ('color', 'black'),
            ('font-size', '16px')
        ]
    }])   
    dfi.export(styled, 'without_context_f1s_overfit_check.png')

In [39]:
whats_with_context2(big_df)

 12%|██████████▌                                                                         | 1/8 [00:00<00:02,  3.22it/s]

['агенс' 'адресат' 'время' 'инструмент' 'место' 'пациенс' 'признак'
 'причина' 'ситуация в фокусе' 'цель' 'экспериенцер']
['агенс' 'адресат' 'время' 'инструмент' 'место' 'пациенс' 'признак'
 'причина' 'цель' 'экспериенцер']


 25%|█████████████████████                                                               | 2/8 [00:00<00:02,  2.43it/s]

['агенс' 'время' 'инструмент' 'место' 'пациенс' 'признак' 'причина'
 'экспериенцер']


 38%|███████████████████████████████▌                                                    | 3/8 [00:01<00:02,  1.90it/s]

...

 75%|███████████████████████████████████████████████████████████████                     | 6/8 [00:06<00:02,  1.38s/it]

['агенс' 'время' 'инструмент' 'место' 'пациенс' 'признак' 'причина']


 88%|█████████████████████████████████████████████████████████████████████████▌          | 7/8 [00:09<00:01,  1.82s/it]

['агенс' 'инструмент' 'место' 'пациенс' 'признак' 'причина']


100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:12<00:00,  1.61s/it]


### без контекста качество лучше

### с оптимальным размером выборки

выборка = 600 

In [10]:
def get_samples(big_df2, num=600):
    counts = pd.DataFrame(big_df2['Role'].value_counts())
    counts.rename(columns={'Role': 'Count'}, inplace=True)
    counts = counts[counts['Count'] > num]
    to_take = list(counts.index)
    res = pd.DataFrame([])
    for role in to_take:
        role_samples = big_df2[big_df2['Role'] == role].sample(n=num, random_state=1)
        res = res.append(role_samples, ignore_index=True)
    return res

In [11]:
more_than_600 = get_samples(big_df)

In [12]:
more_than_600['Role'].unique()

array(['пациенс', 'место', 'агенс', 'инструмент', 'причина', 'признак',
       'время'], dtype=object)

RandomForest importance of feature

In [15]:
enc_features = features(more_than_600, 'Role')

In [16]:
x, y, le_classes = get_X_Y(more_than_600, enc_features)

['агенс' 'время' 'инструмент' 'место' 'пациенс' 'признак' 'причина']


In [17]:
train_x, test_x, train_y, test_y = train_test_split(x, 
                                                    y,
                                                    test_size=0.3, 
                                                    random_state=42)

In [23]:
mod = ensemble.RandomForestClassifier(random_state=3, n_estimators=10, n_jobs=-1)
mod.fit(train_x, train_y)
# model(mod, train_x, train_y, test_x, test_y, le_classes)

RandomForestClassifier(n_estimators=10, n_jobs=-1, random_state=3)

In [13]:
import collections

In [27]:
def check_importance(enc_features, mod):
    importance = mod.feature_importances_
    # summarize feature importance
    important_d = collections.defaultdict(int)


# 'PlaceC', 'Phrase', 'Form', 'Role', 'Rank', 'Last_word', 'pos', 'case',
#        'gender', 'animacy', 'number', 'lemma', 'link_name', 't', 'r',
#        'left_context', 'right_context'
        
    for i,v in enumerate(importance):
        col = enc_features.columns[i] #enc_features_con.columns[i] # 
        if col.startswith('Last_word_'):
            important_d['Last_word'] += v
        elif col.startswith('lemma_'):
            important_d['lemma'] += v
        elif col.startswith('left_context_'):
            important_d['left_context'] += v
        elif col.startswith('right_context_'):
            important_d['right_context'] += v
        elif col.startswith('Role'):
            important_d['Role'] += v
        elif col.startswith('Rank'):
            important_d['Rank'] += v
        elif col.startswith('pos'):
            important_d['pos'] += v
        elif col.startswith('case'):
            important_d['case'] += v
        elif col.startswith('gender'):
            important_d['gender'] += v
        elif col.startswith('animacy'):
            important_d['animacy'] += v
        elif col.startswith('number'):
            important_d['number'] += v
        elif col.startswith('link_name'):
            important_d['link_name'] += v
        elif col.startswith('t'):
            important_d['t'] += v
        elif col.startswith('r'):
            important_d['r'] += v
    return important_d
    
important_d = check_importance(enc_features, mod)
summ = 0
for key in important_d:
    summ += important_d[key]
    print(key, important_d[key])
print(summ)

Last_word 0.04356659495397625
lemma 0.05051845122762426
left_context 0.37992012495088257
right_context 0.3169143141714098
Rank 0.05043072998767556
pos 0.003756221379007031
case 0.03245834009143578
gender 0.008648960587688265
animacy 0.014880497387154417
number 0.008645961200236476
link_name 0.01522398676289037
t 0.048362435808548224
r 0.02667338149147074
0.9999999999999998


importance without context

In [28]:
more_than_600 = more_than_600.drop(['left_context', 'right_context'], axis='columns')

In [29]:
more_than_600.columns

Index(['Role', 'Rank', 'Last_word', 'pos', 'case', 'gender', 'animacy',
       'number', 'lemma', 'link_name', 't', 'r'],
      dtype='object')

In [30]:
enc_features = features(more_than_600, 'Role')

In [31]:
x, y, le_classes = get_X_Y(more_than_600, enc_features)

['агенс' 'время' 'инструмент' 'место' 'пациенс' 'признак' 'причина']


In [32]:
train_x, test_x, train_y, test_y = train_test_split(x, 
                                                    y,
                                                    test_size=0.3, 
                                                    random_state=42)

In [33]:
mod = ensemble.RandomForestClassifier(random_state=3, n_estimators=10, n_jobs=-1)
mod.fit(train_x, train_y)
# model(mod, train_x, train_y, test_x, test_y, le_classes)

RandomForestClassifier(n_estimators=10, n_jobs=-1, random_state=3)

форма слова, его лемма, таксономический класс

In [34]:
important_d = check_importance(enc_features, mod)
summ = 0
for key in important_d:
    summ += important_d[key]
    print(key, important_d[key])
print(summ)

Last_word 0.2682494335407196
lemma 0.2668529717806488
Rank 0.06603136033189422
pos 0.007037305690465135
case 0.057524824784311625
gender 0.03656652074637265
animacy 0.030167227203439013
number 0.02892447146820956
link_name 0.04345552631037105
t 0.14033255489713845
r 0.05485780324642978
1.0


без леммы

In [35]:
more_than_600 = more_than_600.drop(['lemma'], axis='columns')

In [36]:
more_than_600.columns

Index(['Role', 'Rank', 'Last_word', 'pos', 'case', 'gender', 'animacy',
       'number', 'link_name', 't', 'r'],
      dtype='object')

In [70]:
more_than_600.to_csv('more_than_600.tsv', index=False, sep='\t')  

In [37]:
enc_features = features(more_than_600, 'Role')

In [38]:
x, y, le_classes = get_X_Y(more_than_600, enc_features)

['агенс' 'время' 'инструмент' 'место' 'пациенс' 'признак' 'причина']


In [39]:
train_x, test_x, train_y, test_y = train_test_split(x, 
                                                    y,
                                                    test_size=0.3, 
                                                    random_state=42)

In [40]:
mod = ensemble.RandomForestClassifier(random_state=3, n_estimators=10, n_jobs=-1)
mod.fit(train_x, train_y)
# model(mod, train_x, train_y, test_x, test_y, le_classes)

RandomForestClassifier(n_estimators=10, n_jobs=-1, random_state=3)

форма слова, таксономический класс

In [41]:
important_d = check_importance(enc_features, mod)
summ = 0
for key in important_d:
    summ += important_d[key]
    print(key, important_d[key])
print(summ)

Last_word 0.4231605353304962
Rank 0.07156768062969573
pos 0.006785294685876538
case 0.06798778195948318
gender 0.04743637456853142
animacy 0.03765432477020125
number 0.03278880026513062
link_name 0.05998532133915069
t 0.19010101440926258
r 0.06253287204217169
0.9999999999999999


In [43]:
print(metrics.classification_report(test_y, mod.predict(test_x),
                                  target_names=le_classes))

              precision    recall  f1-score   support

       агенс       0.44      0.41      0.42       174
       время       0.81      0.79      0.80       187
  инструмент       0.46      0.52      0.49       178
       место       0.51      0.56      0.53       180
     пациенс       0.48      0.42      0.45       202
     признак       0.32      0.33      0.33       168
     причина       0.29      0.27      0.28       171

    accuracy                           0.48      1260
   macro avg       0.47      0.47      0.47      1260
weighted avg       0.48      0.48      0.48      1260



In [48]:
def big_report(model, name, df_report=pd.DataFrame([])):
#     df_report = pd.DataFrame([])
    
    report = metrics.classification_report(test_y, model.predict(test_x),
                                  target_names=le_classes, output_dict=True)
   
    df_report = pd.concat([df_report, pd.DataFrame(report).transpose().add_prefix(name+'_')[name+'_f1-score']], axis=1)
    return df_report 
    
#     return pd.DataFrame(report).transpose()['f1-score'].add_prefix(name)

In [49]:
rep = big_report(mod, 'RandomForest')

### other models

### knn

In [45]:
def best_k_knn(train_x, train_y, test_x, test_y):
    error_rate = []
    for i in tqdm(range(1,40)):
        knn = KNeighborsClassifier(n_neighbors=i)
        knn.fit(train_x, train_y)
        pred_i = knn.predict(test_x)
        error_rate.append((knn, np.mean(pred_i != test_y)))
    best = sorted(error_rate, key=lambda x: x[1])[0]
    return best[0]

In [46]:
the_mod = best_k_knn(train_x, train_y, test_x, test_y)

100%|██████████████████████████████████████████████████████████████████████████████████| 39/39 [00:26<00:00,  1.46it/s]


best k == 28

In [47]:
print(the_mod)

KNeighborsClassifier(n_neighbors=28)


In [53]:
rep = big_report(the_mod, 'KNeighborsClassifier', rep)

### SVM

In [55]:
from sklearn.svm import SVC

In [56]:
start_time = datetime.now()
svm_classifier = SVC(decision_function_shape='ovr')
svm_classifier.fit(train_x, train_y)

rep = big_report(svm_classifier, 'SVC', rep)


print('Затраченное время:', datetime.now() - start_time)


Затраченное время: 0:00:35.213995


### xgboost

In [58]:
from xgboost import XGBClassifier

In [60]:
from sklearn.multiclass import OneVsRestClassifier

In [61]:
start_time = datetime.now()
xgb_classifier = OneVsRestClassifier(XGBClassifier())
xgb_classifier.fit(train_x, train_y)
rep = big_report(xgb_classifier, 'XGBClassifier', rep)

print('Затраченное время:', datetime.now() - start_time)

D:\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[09:43:12] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[09:43:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[09:43:47] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[09:44:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

In [66]:
rep

,RandomForest_f1-score,KNeighborsClassifier_f1-score,SVC_f1-score,XGBClassifier_f1-score
агенс,0.421365,0.472826,0.564516,0.502994
время,0.802168,0.685446,0.786517,0.788889
инструмент,0.485488,0.444444,0.485549,0.467033
место,0.531579,0.535065,0.544081,0.567237
пациенс,0.447368,0.502674,0.514139,0.463158
признак,0.325581,0.271186,0.289389,0.302469
причина,0.283988,0.270627,0.338109,0.349570
accuracy,0.476190,0.470635,0.508730,0.496032
macro avg,0.471077,0.454610,0.503186,0.491621
weighted avg,0.475438,0.459722,0.507902,0.495577


save report with style

In [68]:
cm = sns.light_palette("green", as_cmap=True)
styled = rep.style.background_gradient(cmap=cm).set_caption("600 sample without context&lemma")\
    .set_table_styles([{
        'selector': 'caption',
        'props': [
            ('color', 'black'),
            ('font-size', '16px')
        ]
    }])
    
dfi.export(styled, '4_models.png')

In [69]:
styled

,RandomForest_f1-score,KNeighborsClassifier_f1-score,SVC_f1-score,XGBClassifier_f1-score
агенс,0.421365,0.472826,0.564516,0.502994
время,0.802168,0.685446,0.786517,0.788889
инструмент,0.485488,0.444444,0.485549,0.467033
место,0.531579,0.535065,0.544081,0.567237
пациенс,0.447368,0.502674,0.514139,0.463158
признак,0.325581,0.271186,0.289389,0.302469
причина,0.283988,0.270627,0.338109,0.349570
accuracy,0.476190,0.470635,0.508730,0.496032
macro avg,0.471077,0.454610,0.503186,0.491621
weighted avg,0.475438,0.459722,0.507902,0.495577


random_forest - best время 0.8

svc - best агенс 0.56 пациенс 0.51 инструмент 0.486

xgb - best место 0.57

macro & weighted avg - лучше всех у svc